In [12]:
import keras
import tensorflow as tf
from keras.applications import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [13]:
train_dir = 'D:/SkinCancerDatasets/dataset/train/'
test_dir = 'D:/SkinCancerDatasets/dataset/test/'

In [14]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

In [15]:
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [16]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=128,
    class_mode="binary",
)

Found 25201 images belonging to 2 classes.


In [17]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=128,
    class_mode="binary",
)

Found 10803 images belonging to 2 classes.


In [18]:
model = InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3),
)

In [19]:
#model.add_layer(Flatten(), inputs=model.input)
#model.add_layer(Dense(1024, activation="relu"))
#model.add_layer(Dropout(0.5))
#model.add_layer(Dense(1, activation="sigmoid"))

x = model.output
#x = Flatten()(x)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
#x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=model.input, outputs=predictions)


In [20]:
optimizer = tf.keras.optimizers.Adam(lr=0.001)
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [21]:
#model.fit_generator(
model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=5,
    validation_data=test_generator,
    validation_steps=len(test_generator),
)

C:\Users\josei\AppData\Local\Temp\ipykernel_7460\2965996069.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/5
197/197 [==============================] - 3883s 20s/step - loss: 0.3979 - accuracy: 0.8275 - val_loss: 0.6197 - val_accuracy: 0.7622
Epoch 2/5
123/197 [=================>............] - ETA: 21:48 - loss: 0.3398 - accuracy: 0.8489

KeyboardInterrupt: 

In [ ]:
score = model.evaluate_generator(
    test_generator, steps=len(test_generator)
)

In [ ]:
y_true, y_pred = test_generator.next()
cm = confusion_matrix(y_true, y_pred)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(cm, cmap="Blues")
plt.title("Matriz de confusión")
plt.xlabel("Clase predicha")
plt.ylabel("Clase real")
plt.xticks([0, 1], ["Clase 0", "Clase 1"])
plt.yticks([0, 1], ["Clase 0", "Clase 1"])
plt.show()